<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE5-WEEK4/m05w04_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ex3:RMSProp

cho một hàm số Loss ban đầu như dưới đây:
$$f(w_1, w_2) = 0.1w_1^2 + 2w_2^2 \;\;\;\;\;\;\;(1)$$ \\
Giá trị khởi tạo:
$$w1 = -5,\; w2 = -2,\; s1 = 0,\; s2 = 0,\; α = 0.3,\; \gamma = 0.9,\; \epsilon = 10^{-6} $$ \\


**Epoch1**:

- **STEP1**: tính đạo hàm riêng cho hàm số \\
$$
\begin{aligned}
{f_{w_1}}^{'} = \frac{\partial f}{\partial w_1} & = 2 \cdot 0.1w_1 = 0.2w_1 = -1  \\
{f_{w_2}}^{'} = \frac{\partial f}{\partial w_2} & = 2 \cdot 2w_2 = 4w_2 = -8
\end{aligned}
$$

- **STEP2**: cập nhật biến số của hàm s \\
$$
\begin{aligned}
s_1^1 &= \gamma \cdot s_1 + (1-\gamma) \cdot ({f_{w_1}}^{'})^2 = 0.9 \cdot 0  + 0.1 \cdot (-1)^2 = 0.1 \\
s_2^1 &= \gamma \cdot s_2 + (1-\gamma) \cdot ({f_{w_2}}^{'})^2 = 0.9 \cdot 0 + 0.1 \cdot (-8)^2 = 6.4
\end{aligned}
$$

- **STEP3**: cập nhật trọng số w \\
$$
\begin{aligned}
w_1^1 &= w_1 - \alpha \cdot \frac{{f_{w_1}}^{'}}{\sqrt[2]{s_1^1 + \epsilon}}  = -5 - 0.3 \cdot \frac{(-1)}{\sqrt[2]{0.1 + 10^{-6}}} = -4.0513  \\
w_2^1 &= w_2 - \alpha \cdot \frac{{f_{w_2}}^{'}}{\sqrt[2]{s_2^1 + \epsilon}}  = -2 - 0.3 \cdot \frac{(-8)}{\sqrt[2]{6.4 + 10^{-6}}} = -1.0513
\end{aligned}
$$ \\


**Epoch2**:

- **STEP1**: tính đạo hàm riêng cho hàm số \\
$$
\begin{aligned}
{f_{w_1}}^{'} = \frac{\partial f}{\partial w_1} & = 2 \cdot 0.1w_1 = 0.2w_1^1 = -0.81026  \\
{f_{w_2}}^{'} = \frac{\partial f}{\partial w_2} & = 2 \cdot 2w_2 = 4w_2^1 = -4.2052
\end{aligned}
$$

- **STEP2**: cập nhật biến số của hàm s \\
$$
\begin{aligned}
s_1^2 &= \gamma \cdot s_1^1 + (1-\gamma) \cdot ({f_{w_1}}^{'})^2  = 0.9 \cdot 0.1 + 0.1 \cdot (-0.81026)^2 = 0.15565  \\
s_2^2 &= \gamma \cdot s_2^1 + (1-\gamma) \cdot ({f_{w_2}}^{'})^2  = 0.9 \cdot 6.4 + 0.1 \cdot (-4.2052)^2 = 5.76
\end{aligned}
$$

- **STEP3**: cập nhật trọng số w \\
$$
\begin{aligned}
w_1^2 &= w_1^1 - \alpha \cdot \frac{{f_{w_1}}^{'}}{\sqrt[2]{s_1^2 + \epsilon}}  = -4.0513 - 0.3 \cdot \frac{(-0.81026)}{\sqrt[2]{0.15565 + 10^{-6}}} = -3.43517  \\
w_2^2 &= w_2^1 - \alpha \cdot \frac{{f_{w_2}}^{'}}{\sqrt[2]{s_1^2 + \epsilon}}  = -1.0513 - 0.3 \cdot \frac{(-4.2052)}{\sqrt[2]{5.76 + 10^{-6}}} = 0.52565
\end{aligned}
$$


In [6]:
import numpy as np

In [7]:
def df_w(W):
    """
    Thực hiện tính gradient của dw1 và dw2
    Arguments:
    W -- np.array [w1, w2]
    Returns:
    dW -- np.array [dw1, dw2], array chứa giá trị đạo hàm theo w1 và w2
    """
    W = np.asarray(W)
    dW = np.array([0.2, 4])*W
    return dW

In [8]:
def RMSProp(W, dW, lr, S, gamma):
    """
    Thực hiện thuật tóan RMSProp để update w1 và w2
    Arguments:
    W -- np.array: [w1, w2]
    dW -- np.array: [dw1, dw2], array chứa giá trị đạo hàm theo w1 và w2
    lr -- float: learning rate
    S -- np.array: [s1, s2] Exponentially weighted averages bình phương gradients
    gamma -- float: hệ số long-range average
    Returns:
    W -- np.array: [w1, w2] w1 và w2 sau khi đã update
    S -- np.array: [s1, s2] Exponentially weighted averages bình phương gradients sau khi đã cập nhật
    """
    epsilon = 1e-6

    S = gamma*S + (1-gamma)*((dW)**2)

    W = W - lr*dW/(np.sqrt(S + epsilon))
    return W, S

In [9]:
def train_p1(optimizer, lr, epochs):
    """
    Thực hiện tìm điểm minimum của function (1) dựa vào thuật toán
    được truyền vào từ optimizer
    Arguments:
    optimize : function thực hiện thuật toán optimization cụ thể
    lr -- float: learning rate
    epochs -- int: số lượng lần (epoch) lặp để tìm điểm minimum
    Returns:
    results -- list: list các cặp điểm [w1, w2] sau mỗi epoch (mỗi lần cập nhật)
    """
    # initial
    W = np.array([-5, -2], dtype=np.float32)
    S = np.array([0, 0], dtype=np.float32)
    results = [W]

    for i in range(epochs):
        dW = df_w(W)
        W, S = optimizer(W, dW, lr, S = S, gamma = 0.9)
        results.append(W)

    return results

In [10]:
train_p1(RMSProp, lr=0.3, epochs=30)

[array([-5., -2.], dtype=float32),
 array([-4.05132145, -1.05131678]),
 array([-3.43519754, -0.59152343]),
 array([-2.95893693, -0.3294394 ]),
 array([-2.56546289, -0.17756482]),
 array([-2.22920552, -0.09163256]),
 array([-1.93626752, -0.04494499]),
 array([-1.67817686, -0.02081423]),
 array([-1.44934985, -0.00903559]),
 array([-1.24588199, -0.00364591]),
 array([-1.06490301, -0.00135351]),
 array([-9.04202260e-01, -4.56444431e-04]),
 array([-7.61996495e-01, -1.37562928e-04]),
 array([-6.36778499e-01, -3.62601019e-05]),
 array([-5.27215237e-01, -8.11337456e-06]),
 array([-4.32078505e-01, -1.47473412e-06]),
 array([-3.50198507e-01, -2.02783991e-07]),
 array([-2.80434649e-01, -1.84231187e-08]),
 array([-2.21659834e-01, -7.67742748e-10]),
 array([-1.72755512e-01,  7.80451998e-12]),
 array([-1.32615134e-01, -5.05794800e-13]),
 array([-1.00153779e-01,  6.19123501e-14]),
 array([-7.43217708e-02, -1.13373781e-14]),
 array([-5.41201278e-02,  2.80166702e-15]),
 array([-3.86159157e-02, -8.81341